## Importing the data

In [1]:
import pandas as pd

energy_data = pd.read_csv("energydata_complete.csv")

In [2]:
#Checking few rows from the data
energy_data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
#Checking all the columns of the data
energy_data.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

# Graded quiz questions

# Question 12
From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [4]:
#Selecting the needed part of the dataframe
df_1 = energy_data[["T2", "T6"]]

#Importing the linear regression library
from sklearn.linear_model import LinearRegression

#separating the feature from the target
features_df_1 = df_1[["T2"]]
Temp_outside = df_1["T6"]

#Now, we split our dataset into the training and testing dataset. Recall that we had earlier segmented the features and target variables.
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features_df_1, Temp_outside, 
                                                    test_size= 0.3 , random_state= 1 )
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train, y_train)

#obtain predictions
predicted_values = linear_model.predict(x_test)

#Using the r2 score to check the performance of the trained model
from sklearn.metrics import r2_score

r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.65

answer 12 = 0.65

# Question 13

Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?

In [5]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

#Dropping the string columns
df_2 = energy_data.drop(columns=[ 'date' , 'lights' ])

scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df_2), columns=df_2.columns)
features_df = normalised_df.drop(columns=["Appliances"])
Target = normalised_df["Appliances"]

#Now, we split our dataset into the training and testing dataset. Recall that we have already segmented the features and target variables.
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features_df, Target, 
                                                    test_size= 0.3 , random_state= 42)
linear_model = LinearRegression()

#fit the model to the training dataset
linear_model.fit(x_train, y_train)

#obtain predictions
predicted_values = linear_model.predict(x_test)

#Obtaining the mean absolut error
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, predicted_values)
round(mae, 2)

0.05

answer 13 = 0.05

# Question 14

What is the Mean Absolute Error (in two decimal places)?

In [6]:
import numpy as np

rss = np.sum(np.square(y_test - predicted_values))
round(rss, 2)

45.35

answer 14 = 45.35

# Question 15

What is the Root Mean Squared Error (in three decimal places)?

In [7]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

answer 15 = 0.088

# Question 16

What is the Coefficient of Determination (in two decimal places)?

In [8]:
from sklearn.metrics import r2_score

r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.15

answer 16 = 0.15

# Question 17

Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [9]:
#Creating a function to get the weights from the trained model
def get_weights_df(model, feat, col_name) :
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, feat.columns, ).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights_df(linear_model, x_train, 'Linear_Model_Weight' )

In [10]:
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [11]:
min_linear_model_weight = linear_model_weights["Linear_Model_Weight"].min()
linear_model_weights[linear_model_weights["Linear_Model_Weight"] == min_linear_model_weight]

,Features,Linear_Model_Weight
0,RH_2,-0.456698


In [12]:
max_linear_model_weight = linear_model_weights["Linear_Model_Weight"].max()
linear_model_weights[linear_model_weights["Linear_Model_Weight"] == max_linear_model_weight]

,Features,Linear_Model_Weight
25,RH_1,0.553547


answer 17 = RH_2 and RH_1

# Question 18

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [13]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(0.4)

#fit the model to the training dataset
ridge_reg.fit(x_train, y_train)

#obtain predictions
predicted_values = ridge_reg.predict(x_test)

#Obtaining the root mean squared error

rmse_ridge = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse_ridge, 3)

0.088

answer 18 = NO

# Question 19

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [14]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(0.001)

#fit the model to the training dataset
lasso_reg.fit(x_train, y_train)

lasso_model_weights = get_weights_df(lasso_reg, x_train, 'Lasso_Model_Weight' )
lasso_model_weights

,Features,Lasso_Model_Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [15]:
lasso_model_weights[lasso_model_weights["Lasso_Model_Weight"] != 0].count()

Features              4
Lasso_Model_Weight    4
dtype: int64

answer 19 = 4

# Question 20

What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

In [16]:
#obtain predictions
predicted_values = lasso_reg.predict(x_test)

#Obtaining the root mean squared error
rmse_lasso = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse_lasso, 3)

0.094

answer 20 = 0.094